# NYU PUI - HOMEWORK 4 - Part 2 Citibikes

Submitted by Zoe Martiniak (zem232)

## Citi Bikes IDEA: 
#### Younger people are more likely to use Citi Bike in the evenings.

## NULL HYPOTHESIS:
#### The proportion of young people (the younger half of the population) is the same or higher than the proportion of the older people (the older half of the population) biking in the evenings (after 7 pm).

# _$H_0$_ : $\frac{Y_{\mathrm{evening}}}{Y_{\mathrm{total}}} - \frac{O_{\mathrm{evening}}}{O_{\mathrm{total}}}>= 0 $
# _$H_1$_ : $\frac{Y_{\mathrm{evening}}}{Y_{\mathrm{total}}} - \frac{O_{\mathrm{evening}}}{O_{\mathrm{total}}} < 0$
#### I will use a significance level  $\alpha=0.05$

In [3]:
"""
Default Imported Packages by Sung Hoon (NetID: shy256)
"""


from __future__ import print_function
__author__ = "Sung Hoon Yang, CUSP NYU 2018"
import numpy as np
import pandas as pd
import matplotlib
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 88}

matplotlib.rc('font', **font)
import matplotlib.pyplot as plt
import seaborn as sns
import os
%matplotlib inline
np.random.seed(999)
import warnings
warnings.filterwarnings(action='once')

"""
Set up PUIDATA env var
"""
import os
os.environ["PUIDATA"] = "%s/fall18/PUI/PUIDATA"% os.getenv("HOME")
os.environ["PUI2018"] = "%s/fall18/PUI/PUI2018"% os.getenv("HOME")

"""
Needed for the notebook
"""
import re
from datetime import datetime
from copy import deepcopy
from pprint import pprint

In [4]:
from __future__  import print_function, division
import pylab as pl
import pandas as pd
import numpy as np
import os
import matplotlib as mp

%pylab inline

Populating the interactive namespace from numpy and matplotlib


/nfshome/shy256/.conda/envs/tf/lib/python3.5/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['datetime']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [5]:
def getCitiBikeCSV(datestring):
    print("Downloading",datestring)
    if not os.path.isfile(os.getenv("PUIDATA") + "/" + datestring + "-citibike-tripdata.zip"):
        if not os.path.isfile(datestring + "-citibike-tripdata.zip"):
            os.system("curl -O https://s3.amazonaws.com/tripdata/" + datestring + "-citibike-tripdata.zip")
        ##  Unzip the folder
        os.system("unzip " + datestring + "-citibike-tripdata.zip")
    ## Move to PUIdata
    os.system("mv " + datestring + "-citibike-tripdata.csv " + os.getenv("PUIDATA"))
    ## Delete zip in directory
    os.system("rm "+ datestring + "-citibike-tripdata.zip")
    ## NOTE: old csv citibike data had a different name structure... and we don't want older data anyway.
    if '2014' in datestring:
        print("Please use a more recent dataset.")
    ### One final check:
    if not os.path.isfile(os.getenv("PUIDATA") + "/" + datestring + "-citibike-tripdata.csv"):
        print ("UH OH! There seems to be something wrong: the file is not there!")
    else:
        print('File is in place, please continue.')

In [6]:
datestring = '201609'
getCitiBikeCSV(datestring)

File is in place, please continue.


In [12]:
cb_df = pd.read_csv(os.getenv("PUIDATA") + "/" + datestring + '-citibike-tripdata.csv')
cb_df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,975,9/1/2016 00:00:02,9/1/2016 00:16:18,312,Allen St & Stanton St,40.722055,-73.989111,313,Washington Ave & Park Ave,40.696102,-73.967510,22609,Subscriber,1985.0,1
1,1220,9/1/2016 00:00:04,9/1/2016 00:20:25,316,Fulton St & William St,40.709560,-74.006536,239,Willoughby St & Fleet St,40.691966,-73.981302,16966,Subscriber,1977.0,2
2,308,9/1/2016 00:00:19,9/1/2016 00:05:27,479,9 Ave & W 45 St,40.760193,-73.991255,448,W 37 St & 10 Ave,40.756604,-73.997901,25601,Subscriber,1983.0,1
3,250,9/1/2016 00:00:25,9/1/2016 00:04:35,297,E 15 St & 3 Ave,40.734232,-73.986923,487,E 20 St & FDR Drive,40.733143,-73.975739,22094,Subscriber,1953.0,1
4,439,9/1/2016 00:00:25,9/1/2016 00:07:45,494,W 26 St & 8 Ave,40.747348,-73.997236,533,Broadway & W 39 St,40.752996,-73.987216,16319,Subscriber,1985.0,1


In [13]:
cb_df.columns

Index(['tripduration', 'starttime', 'stoptime', 'start station id',
       'start station name', 'start station latitude',
       'start station longitude', 'end station id', 'end station name',
       'end station latitude', 'end station longitude', 'bikeid', 'usertype',
       'birth year', 'gender'],
      dtype='object')

In [14]:
## adding age column
cb_df['age']= 2016-cb_df['birth year']
## cleaning datafile
cb_df.drop(['stoptime', 'start station id', 'start station name', 'start station latitude',
        'start station longitude', 'end station id', 'end station name',
        'end station latitude', 'end station longitude', 'bikeid', 'usertype', 
        'birth year'], axis=1, inplace=True)
## removing the trips longer than 8 hours
cb_df=cb_df[cb_df['tripduration']<=480]
## removing people over the age of 105

In [16]:
## adding date and time column
cb_df['date'] = pd.to_datetime(cb_df['starttime'])
cb_df['after_7pm'] = cb_df['date'].apply(lambda dt: dt.hour >= 19)
cb_df.head(7)

,tripduration,starttime,gender,age,date,after_7pm
2,308,9/1/2016 00:00:19,1,33.0,2016-09-01 00:00:19,False
3,250,9/1/2016 00:00:25,1,63.0,2016-09-01 00:00:25,False
4,439,9/1/2016 00:00:25,1,31.0,2016-09-01 00:00:25,False
6,188,9/1/2016 00:00:40,2,28.0,2016-09-01 00:00:40,False
10,186,9/1/2016 00:00:53,1,36.0,2016-09-01 00:00:53,False
16,441,9/1/2016 00:01:04,1,53.0,2016-09-01 00:01:04,False
17,273,9/1/2016 00:01:09,1,73.0,2016-09-01 00:01:09,False


In [18]:
"""
    Making the final data set 
"""
am = cb_df['age'].mean()
# get ix
age_y = cb_df['age']<am
age_o = cb_df['age']>am
# make boolean flag series'
_7pm_a = cb_df['date'].apply(lambda dt: dt.hour >= 19)
_7pm_b = cb_df['date'].apply(lambda dt: dt.hour < 19)

# group by date and then count rows to get trip counts
# total young ppl data = age_y
_tot_y = cb_df[age_y].groupby(cb_df[age_y]['date'].map(lambda x: x.date())).count()
# total old ppl data = age_o
_tot_o = cb_df[age_o].groupby(cb_df[age_o]['date'].map(lambda x: x.date())).count()
# total young ppl data thats later than 7pm = age_y & _7pm_a
_y_7pm_a = cb_df[age_y & _7pm_a].groupby(cb_df[age_y & _7pm_a]['date'].map(lambda x: x.date())).count()
# total old ppl data thats later than 7pm = age_o & _7pm_a 
_o_7pm_a = cb_df[age_o & _7pm_a].groupby(cb_df[age_o & _7pm_a]['date'].map(lambda x: x.date())).count()

In [32]:
_o_7pm_a.head(3)
# since we aggregated by counting, the columns all have the same value - the count.. 
# we will just use after_7pm column only

,tripduration,starttime,gender,age,date,after_7pm
date,,,,,,
2016-09-01,1043,1043,1043,1043,1043,1043
2016-09-02,781,781,781,781,781,781
2016-09-03,669,669,669,669,669,669


In [23]:
# using _df_cnts, merge the dataframe one by one
_df_cnts = pd.DataFrame(index=_tot_y.index)
_columns = _tot_y.columns


for df in ['_tot_y', '_tot_o', '_y_7pm_a', '_o_7pm_a']:
    _df_cnts = _df_cnts.merge(
        globals()[df] # get dataframe from global frame
        , how='inner'
        , left_index=True
        , right_index=True
        , sort=True
        , suffixes=['', df]
    )
    _df_cnts.drop(
        columns=[c for c in _df_cnts.columns if not c.startswith('after_7pm') or c.startswith('cnt')]
        , inplace=True
    )
    _df_cnts.rename(
        columns = {
            'after_7pm_%s' % df: 'cnt_%s' % df 
        }
        , inplace=True
    )

_df_cnts.rename(
        columns = {
            'after_7pm': 'total_young'
            , 'after_7pm_tot_o': 'total_old'
            , 'after_7pm_y_7pm_a': 'total_young_after_7pm'
            , 'after_7pm_o_7pm_a': 'total_old_after_7pm'
        }
        , inplace=True
    )
_df_cnts.head(5)

,total_young,total_old,total_young_after_7pm,total_old_after_7pm
date,,,,
2016-09-01,8644,5693,2209,1043
2016-09-02,9703,6620,1544,781
2016-09-03,5786,3767,1137,669
2016-09-04,5480,3695,1230,624
2016-09-05,6974,4369,1579,758


In [24]:
_df_cnts['young_proportion'] = _df_cnts['total_young_after_7pm'] / _df_cnts['total_young']
_df_cnts['old_proportion'] = _df_cnts['total_old_after_7pm'] / _df_cnts['total_old']

In [25]:
_columns_i_want = ['young_proportion', 'old_proportion']
_df_cnts.drop(columns=[c for c in _df_cnts.columns if c not in _columns_i_want], inplace=True)
_df_cnts.head(3)

,young_proportion,old_proportion
date,,
2016-09-01,0.255553,0.183207
2016-09-02,0.159126,0.117976
2016-09-03,0.196509,0.177595


#### I am adding the following clause to the statement of the hypothesis test:
##### Original Statement: The proportion of young people (the younger half of the population) is the same or higher than the proportion of the older people (the older half of the population) biking in the evenings (after 7 pm).
##### Modified Statement: The *daily* proportion of young people (the younger half of the population) is the same or higher than the proportion of the older people (the older half of the population) biking in the evenings (after 7 pm).

# _$H_0$_ : $\frac{Y_{\mathrm{evening}}}{Y_{\mathrm{total}}} - \frac{O_{\mathrm{evening}}}{O_{\mathrm{total}}}<= 0 $
# _$H_1$_ : $\frac{Y_{\mathrm{evening}}}{Y_{\mathrm{total}}} - \frac{O_{\mathrm{evening}}}{O_{\mathrm{total}}} > 0$

#### My test will be conducted with **95% significance level**
Since we have only 30 samples, I will be performing t-test for the difference between means of two samples

In [26]:
DOF = len(_df_cnts) - 1
mean_young = _df_cnts['young_proportion'].mean()
std_young = _df_cnts['young_proportion'].std()
mean_old = _df_cnts['old_proportion'].mean()
std_old = _df_cnts['old_proportion'].std()
"""DOF: %d | mean_young: %.2f | std_young: %.2f | mean_old: %.2f | std_old: %.2f""" % (DOF, mean_young, std_young, mean_old, std_old)

'DOF: 29 | mean_young: 0.21 | std_young: 0.03 | mean_old: 0.15 | std_old: 0.02'

In [27]:
se = np.sqrt(
    (std_young ** 2 / len(_df_cnts)) +
    (std_old ** 2 / len(_df_cnts))
)
se

0.006805934942306373

In [31]:
#### for 95% confidence test,
t_stat = 1.311434

t_stat_observed = (mean_young - mean_old) / se
t_stat_observed

9.011979224038146

#### Since 9.011979224038146 > 1.311434, we reject null hypothesis at 95% confidence level

In [ ]:
## End of Notebook 